In [6]:
import sys
sys.version

'3.5.2 (default, Nov 23 2017, 16:37:01) \n[GCC 5.4.0 20160609]'

In [7]:
#basics
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',500)

pd.set_option('display.max_rows', 300)

#files 
import pickle

#plots
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from imblearn.over_sampling import RandomOverSampler 
from sklearn.naive_bayes import MultinomialNB

In [3]:
df = pd.read_pickle('prev_lightning_fires.pkl')
df.shape

(1880465, 20)

In [8]:
df.index

Int64Index([      2,       4,       5,      25,      27,      28,      40,
                 48,      55,      56,
            ...
            1547652, 1547872, 1548310, 1548409, 1548750, 1549990, 1550736,
            1599975, 1759854, 1770047],
           dtype='int64', length=1880465)

In [9]:
df.columns

Index(['FOD_ID', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME',
       'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'FIRE_SIZE', 'FIRE_SIZE_CLASS',
       'LATITUDE', 'LONGITUDE', 'STATE', 'Month', 'DayofWeek',
       'DISCOVERY_TIME2', 'COUNTY2_x', 'COUNTY_ID', 'CLASS', 'StateID',
       'CAUSE', 'Prev_Lightning_Fires'],
      dtype='object')

In [10]:
df['DISCOVERY_DATE'][0]

Timestamp('1992-01-01 00:00:00')

In [11]:
#use
#generalize 
def prev_fires_filter(group, time_col, value_col, condition, new_col_name, year_interval=5):
    for idx, row in group.iterrows():
        past = (group[time_col] < row[time_col]) & (group[value_col] == condition)
        df = group[past] #make dataframe of past fires relative to current observation
        
        #df['yr_intervals'] = (row[time_col] - df[time_col]) // year_interval #if using 'FIRE_YEAR' col
        #print(df['yr_intervals'])
        df['yr_intervals'] = (row[time_col] - df[time_col]) // np.timedelta64(year_interval, 'Y') #how many 5-years have passed?
        #print(df['yr_intervals'])
        df['weight'] = 1 / 2**(df['yr_intervals'])
                #weight of 1 for within past 5 years, 0.5 for 5-10 years, 0.25 for 10-15 years, etc
        #df['prev_fires'] = df['weight'].sum()
        #print(df['weight'])
        count_fires = df['weight'].sum()
        #print(count_fires)
        #count_fires = df['prev_fires'].iloc[0]
        group.loc[idx,new_col_name] = count_fires
    return group
        

In [12]:
df.head()

,FOD_ID,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,Month,DayofWeek,DISCOVERY_TIME2,COUNTY2_x,COUNTY_ID,CLASS,StateID,CAUSE,Prev_Lightning_Fires
2,1127038,1992-01-01,1,None,5.0,Debris Burning,5.0,B,34.391700,-78.568300,NC,1,Wednesday,NaN,columbus,37047,1,37,Accident,0.0
4,250183,1992-01-01,1,1234,9.0,Miscellaneous,20.0,C,25.882760,-80.959320,FL,1,Wednesday,1234.0,collier,12021,2,12,Other,0.0
5,846643,1992-01-01,1,None,13.0,Missing/Undefined,0.2,A,34.258333,-80.495833,SC,1,Wednesday,NaN,kershaw,45055,1,45,Other,0.0
25,1130895,1992-01-01,1,None,5.0,Debris Burning,1.0,B,36.208300,-81.295000,NC,1,Wednesday,NaN,wilkes,37193,1,37,Accident,0.0
27,846971,1992-01-01,1,None,13.0,Missing/Undefined,4.0,B,33.920833,-80.279167,SC,1,Wednesday,NaN,sumter,45085,1,45,Other,0.0


In [13]:

%time df = df.groupby(['COUNTY_ID']).apply(prev_fires_filter, 'DISCOVERY_DATE', 'CAUSE', 'Accident', 'Prev_Accident_Fires')

CPU times: user 2h 28min 15s, sys: 1.78 s, total: 2h 28min 17s
Wall time: 2h 28min 14s


In [14]:
%time df = df.groupby(['COUNTY_ID']).apply(prev_fires_filter, 'DISCOVERY_DATE', 'CAUSE', 'Arson', 'Prev_Arson_Fires')

CPU times: user 2h 26min 54s, sys: 1.87 s, total: 2h 26min 56s
Wall time: 2h 26min 53s


In [15]:
%time df = df.groupby(['COUNTY_ID']).apply(prev_fires_filter, 'DISCOVERY_DATE', 'CAUSE', 'Other', 'Prev_Other_Fires')

CPU times: user 2h 27min 3s, sys: 1.64 s, total: 2h 27min 5s
Wall time: 2h 27min 2s


In [16]:
df.head()

,FOD_ID,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,Month,DayofWeek,DISCOVERY_TIME2,COUNTY2_x,COUNTY_ID,CLASS,StateID,CAUSE,Prev_Lightning_Fires,Prev_Accident_Fires,Prev_Arson_Fires,Prev_Other_Fires
2,1127038,1992-01-01,1,None,5.0,Debris Burning,5.0,B,34.391700,-78.568300,NC,1,Wednesday,NaN,columbus,37047,1,37,Accident,0.0,0.0,0.0,0.0
4,250183,1992-01-01,1,1234,9.0,Miscellaneous,20.0,C,25.882760,-80.959320,FL,1,Wednesday,1234.0,collier,12021,2,12,Other,0.0,0.0,0.0,0.0
5,846643,1992-01-01,1,None,13.0,Missing/Undefined,0.2,A,34.258333,-80.495833,SC,1,Wednesday,NaN,kershaw,45055,1,45,Other,0.0,0.0,0.0,0.0
25,1130895,1992-01-01,1,None,5.0,Debris Burning,1.0,B,36.208300,-81.295000,NC,1,Wednesday,NaN,wilkes,37193,1,37,Accident,0.0,0.0,0.0,0.0
27,846971,1992-01-01,1,None,13.0,Missing/Undefined,4.0,B,33.920833,-80.279167,SC,1,Wednesday,NaN,sumter,45085,1,45,Other,0.0,0.0,0.0,0.0


In [18]:
df.to_pickle('historic_by_cause_done.pkl')

In [4]:
fires = pd.read_pickle('historic_by_cause_done.pkl')

In [5]:
fires.head()

,FOD_ID,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,Month,DayofWeek,DISCOVERY_TIME2,COUNTY2_x,COUNTY_ID,CLASS,StateID,CAUSE,Prev_Lightning_Fires,Prev_Accident_Fires,Prev_Arson_Fires,Prev_Other_Fires
2,1127038,1992-01-01,1,None,5.0,Debris Burning,5.0,B,34.391700,-78.568300,NC,1,Wednesday,NaN,columbus,37047,1,37,Accident,0.0,0.0,0.0,0.0
4,250183,1992-01-01,1,1234,9.0,Miscellaneous,20.0,C,25.882760,-80.959320,FL,1,Wednesday,1234.0,collier,12021,2,12,Other,0.0,0.0,0.0,0.0
5,846643,1992-01-01,1,None,13.0,Missing/Undefined,0.2,A,34.258333,-80.495833,SC,1,Wednesday,NaN,kershaw,45055,1,45,Other,0.0,0.0,0.0,0.0
25,1130895,1992-01-01,1,None,5.0,Debris Burning,1.0,B,36.208300,-81.295000,NC,1,Wednesday,NaN,wilkes,37193,1,37,Accident,0.0,0.0,0.0,0.0
27,846971,1992-01-01,1,None,13.0,Missing/Undefined,4.0,B,33.920833,-80.279167,SC,1,Wednesday,NaN,sumter,45085,1,45,Other,0.0,0.0,0.0,0.0


In [15]:
def prev_fires_filter(group, time_col, value_col, condition, new_col_name, year_interval=5):
    for idx, row in group.iterrows():
        if idx%10000 == 0:
            print(idx)
        past = (group[time_col] < row[time_col]) & (group[value_col] == condition)
        df = group[past] #make dataframe of past fires relative to current observation
        
        df['yr_intervals'] = (row[time_col] - df[time_col]) // np.timedelta64(year_interval, 'Y') #how many 5-years have passed?
        
        df['weight'] = 1 / 2**(df['yr_intervals'])
                #weight of 1 for within past 5 years, 0.5 for 5-10 years, 0.25 for 10-15 years, etc
        count_fires = df['weight'].sum()
        group.loc[idx,new_col_name] = count_fires
    return group


In [16]:
#use to get Prev_fires at location
def prev_fires(group, time_col, new_col_name, year_interval=5):
    for idx, row in group.iterrows():
        if idx%10000 == 0:
            print(idx) 
        past = group[time_col] < row[time_col]
        df = group[past] #make dataframe of past fires relative to current observation
        
        #df['yr_intervals'] = (row[time_col] - df[time_col]) // year_interval #if using 'FIRE_YEAR' col
        #print(df['yr_intervals'])
        df['yr_intervals'] = (row[time_col] - df[time_col]) // np.timedelta64(year_interval, 'Y') #how many 5-years have passed?
        df['weight'] = 1 / 2**(df['yr_intervals']) 
                #weight of 1 for within past 5 years, 0.5 for 5-10 years, 0.25 for 10-15 years, etc
        #df['prev_fires'] = df['weight'].sum()
        #print(df['prev_fires'])
        count_fires = df['weight'].sum()
        #print(count_fires)
        #count_fires = df['prev_fires'].iloc[0]
        group.loc[idx,new_col_name] = count_fires
    return group


        

In [20]:
%time fires = fires.groupby(['COUNTY_ID']).apply(prev_fires, 'DISCOVERY_DATE', 'Prev_Fires_at_Location')
# may be kind of off without the history - ideally use full data set

CPU times: user 9h 13min 29s, sys: 13.5 s, total: 9h 13min 42s
Wall time: 9h 20min 29s


In [24]:
fires.to_pickle('historic_by_location.pkl')

In [30]:
len(fires[fires['Month'] == 7])

237234

In [31]:
#break up the fires
for i in range(1,13):
    fires[fires['Month'] == i].to_pickle('firesbymonth%s.pkl' % str(i))
    

In [4]:
fires = pd.read_pickle('historic_by_location.pkl')

In [5]:
%time fires = fires.groupby(['Month']).apply(prev_fires, 'DISCOVERY_DATE', 'Prev_Fires_Same_Month')

CPU times: user 11h 37min 38s, sys: 2.52 s, total: 11h 37min 41s
Wall time: 11h 37min 38s


In [14]:
fires[fires['Month'] == 11]

,FOD_ID,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,Month,DayofWeek,DISCOVERY_TIME2,COUNTY2_x,COUNTY_ID,CLASS,StateID,CAUSE,Prev_Lightning_Fires,Prev_Accident_Fires,Prev_Arson_Fires,Prev_Other_Fires,Prev_Fires_at_Location,Prev_Fires_Same_Month,Prev_1_fires2
65507,45846,1992-11-01,306,1230,4.0,Campfire,0.10,A,43.058056,-110.851667,WY,11,Sunday,1230.0,lincoln,56023,1,56,Accident,30.0000,17.0000,0.0000,2.0000,49.0000,0.0000,0.000000
65513,759809,1992-11-01,306,None,3.0,Smoking,0.10,A,38.361340,-105.759560,CO,11,Sunday,NaN,fremont,08043,1,08,Accident,12.0000,16.0000,0.0000,6.0000,34.0000,0.0000,0.000000
65514,1387204,1992-11-01,306,None,2.0,Equipment Use,0.10,A,33.693056,-116.188889,CA,11,Sunday,NaN,riverside,06065,1,06,Accident,39.0000,438.0000,87.0000,328.0000,892.0000,0.0000,0.000000
65516,1373565,1992-11-01,306,None,2.0,Equipment Use,0.10,A,34.026944,-117.076944,CA,11,Sunday,NaN,san bernardino,06071,1,06,Accident,65.0000,298.0000,51.0000,130.0000,544.0000,0.0000,0.000000
65518,840033,1992-11-01,306,None,13.0,Missing/Undefined,0.30,B,33.891667,-79.112500,SC,11,Sunday,NaN,horry,45051,1,45,Other,0.0000,0.0000,0.0000,105.0000,105.0000,0.0000,0.000000
65520,198671,1992-11-01,306,1409,5.0,Debris Burning,0.10,A,42.996100,-108.669000,WY,11,Sunday,1409.0,fremont,56013,1,56,Accident,5.0000,47.0000,2.0000,4.0000,58.0000,0.0000,0.000000
65523,840032,1992-11-01,306,None,13.0,Missing/Undefined,2.00,B,33.904167,-79.108333,SC,11,Sunday,NaN,horry,45051,1,45,Other,0.0000,0.0000,0.0000,105.0000,105.0000,0.0000,0.000000
65528,841564,1992-11-01,306,None,13.0,Missing/Undefined,1.40,B,33.341667,-80.279167,SC,11,Sunday,NaN,orangeburg,45075,1,45,Other,0.0000,0.0000,0.0000,175.0000,175.0000,0.0000,0.000000
65531,1394321,1992-11-01,306,None,2.0,Equipment Use,10.00,C,35.076944,-120.531944,CA,11,Sunday,NaN,san luis obispo,06079,1,06,Accident,0.0000,110.0000,19.0000,33.0000,162.0000,0.0000,0.000000
65532,242955,1992-11-01,306,1200,4.0,Campfire,0.10,A,36.048190,-112.116660,AZ,11,Sunday,1200.0,coconino,04005,1,04,Accident,398.0000,193.0000,6.0000,21.0000,618.0000,0.0000,0.000000


In [17]:
%time fires = fires.groupby(['Month']).apply(prev_fires_filter, 'DISCOVERY_DATE', 'CLASS', 2, 'Prev_2_Fires')

670000
900000
1060000
0
130000
280000
970000
1270000
1740000
1810000
670000
900000
1060000
0
130000
280000
970000
1270000
1740000
1810000
290000
580000
10000
70000
210000
490000
680000
760000
1070000
1180000
1360000
1520000
140000
770000
1090000
1190000
1440000
1530000
150000
220000
300000
360000
420000
500000
590000
840000
910000
980000
1080000
1280000
1370000
1610000
1680000
1750000
1820000
80000
230000
510000
780000
990000
1290000
20000
310000
370000
430000
600000
690000
850000
920000
930000
1000000
1100000
1200000
1380000
1390000
1450000
1540000
1620000
1690000
1760000
1830000
90000
380000
520000
700000
790000
1460000
1630000
1700000
30000
160000
610000
1110000
1210000
1300000
1550000
1770000
1840000
40000
170000
240000
320000
440000
620000
710000
860000
940000
1220000
1310000
1400000
800000
1010000
1120000
1560000
50000
100000
180000
330000
390000
450000
530000
630000
810000
1230000
1470000
1640000
1850000
250000
720000
870000
950000
1020000
1130000
1140000
1320000
1410000
1570000

In [18]:
%time fires = fires.groupby(['Month']).apply(prev_fires_filter, 'DISCOVERY_DATE', 'CLASS', 3, 'Prev_3_fires')

670000
900000
1060000
0
130000
280000
970000
1270000
1740000
1810000
670000
900000
1060000
0
130000
280000
970000
1270000
1740000
1810000
290000
580000
10000
70000
210000
490000
680000
760000
1070000
1180000
1360000
1520000
140000
770000
1090000
1190000
1440000
1530000
150000
220000
300000
360000
420000
500000
590000
840000
910000
980000
1080000
1280000
1370000
1610000
1680000
1750000
1820000
80000
230000
510000
780000
990000
1290000
20000
310000
370000
430000
600000
690000
850000
920000
930000
1000000
1100000
1200000
1380000
1390000
1450000
1540000
1620000
1690000
1760000
1830000
90000
380000
520000
700000
790000
1460000
1630000
1700000
30000
160000
610000
1110000
1210000
1300000
1550000
1770000
1840000
40000
170000
240000
320000
440000
620000
710000
860000
940000
1220000
1310000
1400000
800000
1010000
1120000
1560000
50000
100000
180000
330000
390000
450000
530000
630000
810000
1230000
1470000
1640000
1850000
250000
1020000
1130000
1140000
1320000
1410000
1570000
1710000
1780000
1100

In [20]:
fires.tail()

,FOD_ID,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,Month,DayofWeek,DISCOVERY_TIME2,COUNTY2_x,COUNTY_ID,CLASS,StateID,CAUSE,Prev_Lightning_Fires,Prev_Accident_Fires,Prev_Arson_Fires,Prev_Other_Fires,Prev_Fires_at_Location,Prev_Fires_Same_Month,Prev_1_fires2,Prev_2_Fires,Prev_3_fires
1549990,201158095,2011-05-29,149,1608,1.0,Lightning,1.00,B,31.833437,-81.304325,GA,5,Sunday,1608.0,bryan south,13030,1,13,Lightning,6.75,64.375,45.625,13.375,130.125,64387.8750,23624.875000,5630.2500,351.8750
1550736,201158096,2011-06-01,152,1803,5.0,Debris Burning,0.35,B,31.908335,-81.375782,GA,6,Wednesday,1803.0,bryan south,13030,1,13,Accident,7.75,64.375,45.625,13.375,131.125,67760.0000,24772.548828,7088.0000,745.5000
1599975,201158097,2011-12-22,356,0611,7.0,Arson,5.36,B,31.802610,-81.235682,GA,12,Thursday,611.0,bryan south,13030,1,13,Arson,7.25,62.875,42.500,12.625,125.250,25849.8750,10426.714844,3320.8750,53.2500
1759854,300006799,2014-04-06,96,1330,4.0,Campfire,0.10,A,56.997778,-133.317778,AK,4,Sunday,1330.0,wrangell-petersburg,02280,1,02,Accident,0.00,0.000,0.000,0.000,0.000,96311.4375,32234.894531,12236.0625,368.8125
1770047,300006802,2014-05-10,130,1300,7.0,Arson,2.68,B,55.479167,-133.108333,AK,5,Saturday,1300.0,prince of wales-outer ketchikan,02201,1,02,Arson,0.00,0.000,0.000,0.000,0.000,64255.4375,24238.087402,5774.0625,373.2500


In [21]:
fires.to_pickle('historic_by_month_done.pkl')

In [ ]:
#### Not ran below #####

In [ ]:
fires.sample(10)

In [23]:
len(fires[fires['COUNTY_ID'] == '45015'])

5522

# Hold out data sets

In [6]:
newest = fires.tail(150000)
recents = newest.sample(10000)
recents.to_pickle('full_recent_samp.pkl')

In [8]:
remaining = fires[~fires['FOD_ID'].isin(recents['FOD_ID'])]
evals = remaining.sample(10000)
evals.to_pickle('full_eval_samp.pkl')

In [9]:
train = remaining[~remaining['FOD_ID'].isin(evals['FOD_ID'])]
train.to_pickle('full_training_fires.pkl')

In [10]:
train.shape

(1860465, 23)

# Modeling

In [13]:
train['CLASS'].value_counts()

1    1623727
2     225738
3      11000
Name: CLASS, dtype: int64

In [14]:
fires['CLASS'].value_counts()

1    1641510
2     227868
3      11087
Name: CLASS, dtype: int64

Only 0.4% in test set are class 3. Class 3 makes up 0.6% of the training set.

In [3]:
def cv_features(model, X, y):
    '''Inputs:
    model = initialized model
    X = features as an np.array
    y = target as an np.array
    
    Output: 
    Tuple of f1, precision, and recall scores averaged across 5 cross validation folds
    
    Prints:
    Confusion matrix and classfication report for last cross validation fold
    
    Performs a stratified cross validation .
    
    '''
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)
    fs = []
    ps = []
    rs = []
    classes = [1, 2, 3]

    for train_ind, val_ind in kf.split(X, y): # stratify and split by those indices
        model.fit(X[train_ind],y[train_ind])
        model_preds = model.predict(X[val_ind])
        fs.append(f1_score(y[val_ind], model_preds, labels=classes, average=None))
        ps.append(precision_score(y[val_ind], model_preds, labels=classes, average=None))
        rs.append(recall_score(y[val_ind], model_preds, labels=classes, average=None))
        
    print(confusion_matrix(y[val_ind], model_preds, labels=classes)) # print for last one
    print(classification_report(y[val_ind], model_preds, labels=classes))
    print('f1 -- precision -- recall')
    return np.mean(fs, axis=0), np.mean(ps, axis=0), np.mean(rs, axis=0)

In [4]:
fires = pd.read_pickle('full_training_fires.pkl')

# Model A: Establish baseline with latitude/longitude

In [5]:
x1 = np.array(fires[['LATITUDE', 'LONGITUDE']])
y1 = np.array(fires['CLASS'])

In [6]:
lr1 = LogisticRegression(C=100000, solver='sag', n_jobs=14)
cv_features(lr1, x1, y1)

[[324745      0      0]
 [ 45147      0      0]
 [  2200      0      0]]
             precision    recall  f1-score   support

          1       0.87      1.00      0.93    324745
          2       0.00      0.00      0.00     45147
          3       0.00      0.00      0.00      2200

avg / total       0.76      0.87      0.81    372092

f1 -- precision -- recall


(array([0.93205369, 0.        , 0.        ]),
 array([0.87275332, 0.        , 0.        ]),
 array([1., 0., 0.]))

In [9]:
weights={1:1, 2:12, 3:100}
lr1b = LogisticRegression(C=100000, class_weight=weights, n_jobs=-1)
cv_features(lr1b, x1, y1)

[[322828      0   1917]
 [ 45056      0     91]
 [  2189      0     11]]
             precision    recall  f1-score   support

          1       0.87      0.99      0.93    324745
          2       0.00      0.00      0.00     45147
          3       0.01      0.01      0.01      2200

avg / total       0.76      0.87      0.81    372092

f1 -- precision -- recall


(array([0.92928747, 0.        , 0.00655573]),
 array([0.87236049, 0.        , 0.00686629]),
 array([0.99416281, 0.        , 0.00627273]))

In [ ]:
rf1 = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
%time cv_features(rf1, x1,y1)

In [ ]:

rf1b = RandomForestClassifier(n_estimators=1000, class_weight=weights, n_jobs=-1)
%cv_features(rf1b,x1,y1) #best among three

In [10]:
weights={1:1, 2:12, 3:100}

In [4]:
x4 = np.array(fires[['Month', 'LATITUDE', 'LONGITUDE', 'Prev_Lightning_Fires','Prev_Accident_Fires',
       'Prev_Arson_Fires', 'Prev_Other_Fires']])
y4 = np.array(fires['CLASS'])


In [12]:
rf4 = RandomForestClassifier(n_estimators=1000, n_jobs=14, class_weight=weights, max_depth=3, min_samples_leaf=2)
%time cv_features(rf4, x4, y4)

[[ 70532 210936  43277]
 [  2928  37513   4706]
 [   293    849   1058]]
             precision    recall  f1-score   support

          1       0.96      0.22      0.35    324745
          2       0.15      0.83      0.25     45147
          3       0.02      0.48      0.04      2200

avg / total       0.85      0.29      0.34    372092

f1 -- precision -- recall
CPU times: user 3h 22min 41s, sys: 16.9 s, total: 3h 22min 58s
Wall time: 51min 32s


(array([0.35283752, 0.25500266, 0.04241639]),
 array([0.95640944, 0.15058046, 0.0221563 ]),
 array([0.21632762, 0.83189362, 0.49563636]))

(1860465, 23)

In [7]:
#try weaker class weights
x5 = np.array(fires[['Month', 'LATITUDE', 'LONGITUDE', 'Prev_Lightning_Fires','Prev_Accident_Fires',
       'Prev_Arson_Fires', 'Prev_Other_Fires']])
y5 = np.array(fires['CLASS'])
weights={1:1, 2:5, 3:41}
rf5 = RandomForestClassifier(n_estimators=1000, n_jobs=14, class_weight=weights, max_depth=3, min_samples_leaf=2)
%time cv_features(rf5, x5, y5)

[[297745  26457    543]
 [ 34822  10076    249]
 [  1890    138    172]]
             precision    recall  f1-score   support

          1       0.89      0.92      0.90    324745
          2       0.27      0.22      0.25     45147
          3       0.18      0.08      0.11      2200

avg / total       0.81      0.83      0.82    372092

f1 -- precision -- recall
CPU times: user 1h 46min 53s, sys: 1min 29s, total: 1h 48min 22s
Wall time: 27min 12s


(array([0.90366841, 0.24333439, 0.11568354]),
 array([0.88986515, 0.2735471 , 0.19396667]),
 array([0.91790923, 0.21917448, 0.08245455]))

In [9]:
fires.sample(5)

,FOD_ID,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,Month,DayofWeek,DISCOVERY_TIME2,COUNTY2_x,COUNTY_ID,CLASS,StateID,CAUSE,Prev_Lightning_Fires,Prev_Accident_Fires,Prev_Arson_Fires,Prev_Other_Fires
907830,631390,2004-03-04,64,None,5.0,Debris Burning,1.00,B,33.410830,-80.655000,SC,3,Thursday,NaN,orangeburg,45075,1,45,Accident,29.00,1001.5,206.000,506.00
322813,93916,1996-06-26,178,1345,1.0,Lightning,2.00,B,35.408333,-105.608333,NM,6,Wednesday,1345.0,san miguel,35047,1,35,Lightning,46.00,16.0,0.000,160.00
1304054,736986,2008-05-18,139,None,7.0,Arson,0.25,A,45.661980,-92.718440,WI,5,Sunday,NaN,burnett,55013,1,55,Arson,40.75,271.0,37.125,137.25
1204165,1674614,2007-04-23,113,0915,5.0,Debris Burning,4.00,B,38.410330,-83.109470,KY,4,Monday,915.0,carter,21043,1,21,Accident,0.50,52.0,56.750,194.50
1654828,201600871,2012-08-16,229,None,9.0,Miscellaneous,20.00,C,43.545189,-97.132702,SD,8,Thursday,NaN,mcpherson,46089,2,46,Other,11.75,55.0,0.000,14.25


In [14]:
other_stats = pd.read_pickle('historic_by_month_done.pkl')

In [16]:
other_stats.sample(5)

,FOD_ID,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,Month,DayofWeek,DISCOVERY_TIME2,COUNTY2_x,COUNTY_ID,CLASS,StateID,CAUSE,Prev_Lightning_Fires,Prev_Accident_Fires,Prev_Arson_Fires,Prev_Other_Fires,Prev_Fires_at_Location,Prev_Fires_Same_Month,Prev_1_fires2,Prev_2_Fires,Prev_3_fires
898330,650385,2003-12-26,360,None,5.0,Debris Burning,3.00,B,32.923050,-94.626950,TX,12,Friday,NaN,cass,48067,1,48,Accident,32.25,312.25,142.00,102.75,589.25,19103.25,8427.562500,2238.00,28.25
1082441,619379,2006-03-10,69,None,8.0,Children,3.00,B,34.777370,-82.613690,SC,3,Friday,NaN,pickens,45077,1,45,Accident,9.00,265.50,19.50,161.25,455.25,83277.75,35291.265625,14824.00,155.50
192940,783074,1994-08-28,240,1545,3.0,Smoking,2.00,B,33.937309,-89.640786,MS,8,Sunday,1545.0,yalobusha,28161,1,28,Accident,2.00,66.00,62.00,3.00,133.00,22961.00,17697.000000,1957.00,221.00
904849,463193,2004-02-21,52,None,5.0,Debris Burning,1.47,B,32.469969,-83.153732,GA,2,Saturday,NaN,laurens,13175,1,13,Accident,70.50,1249.25,172.25,84.00,1576.00,52464.75,19297.578125,8734.75,89.00
926159,917932,2004-04-16,107,None,5.0,Debris Burning,1.00,B,31.959200,-89.598100,MS,4,Friday,NaN,smith,28129,1,28,Accident,1.50,164.25,175.50,109.50,450.75,76881.50,32931.750000,10076.75,179.75


In [9]:
evals = pd.read_pickle('full_eval_samp.pkl')
recents = pd.read_pickle('full_recent_samp.pkl')

In [7]:
#filter out holdout sets
remaining2 = other_stats[~other_stats['FOD_ID'].isin(recents['FOD_ID'].append(evals['FOD_ID']))]

In [12]:
len(remaining2)

1860465

In [9]:
#grab columns not in other df 
#remaining2.iloc[:,-5:]

In [9]:
combined = pd.concat([fires, remaining2.iloc[:,-5:]], axis=1)
combined.shape

(1860465, 28)

In [11]:
combined.sample(10)

,FOD_ID,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,Month,DayofWeek,DISCOVERY_TIME2,COUNTY2_x,COUNTY_ID,CLASS,StateID,CAUSE,Prev_Lightning_Fires,Prev_Accident_Fires,Prev_Arson_Fires,Prev_Other_Fires,Prev_Fires_at_Location,Prev_Fires_Same_Month,Prev_1_fires2,Prev_2_Fires,Prev_3_fires
903452,201921825,2004-02-11,42,None,13.0,Missing/Undefined,1.00,B,18.363047,-67.186050,PR,2,Wednesday,NaN,aguada,72003,1,72,Other,0.000,0.00,0.00,54.000,54.00,53310.750,21169.093750,8901.000,87.500
312021,19910828,1996-05-04,125,0900,5.0,Debris Burning,14.00,C,33.112800,-85.835390,AL,5,Saturday,900.0,clay,01027,2,01,Accident,4.000,142.00,30.00,2.000,178.00,27843.000,18040.000000,2537.000,87.000
1067405,201928213,2006-02-04,35,None,13.0,Missing/Undefined,0.25,A,18.108768,-67.046249,PR,2,Saturday,NaN,san german,72125,1,72,Other,0.000,0.00,0.00,397.000,397.00,51857.750,18991.671875,8585.000,79.750
497441,1162167,1999-03-18,77,None,5.0,Debris Burning,6.00,B,34.971700,-79.095000,NC,3,Thursday,NaN,hoke,37093,1,37,Accident,2.500,269.00,296.00,25.500,593.00,54676.000,25616.625000,9795.000,91.000
845557,1184733,2003-04-03,93,None,2.0,Equipment Use,0.50,B,35.666700,-75.866700,NC,4,Thursday,NaN,dare,37055,1,37,Accident,14.000,516.75,13.00,24.500,568.25,70820.250,31518.937500,9379.750,162.000
1359512,1439896,2009-02-06,37,None,5.0,Debris Burning,10.00,C,40.337124,-96.688957,NE,2,Friday,NaN,gage,31067,1,31,Accident,1.500,43.00,1.00,11.500,57.00,60579.750,23031.148438,10285.875,151.250
1035595,1818057,2005-09-19,262,1623,9.0,Miscellaneous,0.10,A,40.567811,-74.112247,NY,9,Monday,1623.0,richmond,36085,1,36,Other,117.000,49.75,32.50,664.750,864.00,45140.000,18348.421875,4293.000,238.750
442313,111682,1998-06-23,174,1636,1.0,Lightning,0.30,B,44.467500,-120.723333,OR,6,Tuesday,1636.0,crook,41013,1,41,Lightning,370.500,108.00,0.00,25.500,504.00,34498.500,18288.875000,3326.500,345.500
1607407,201701635,2012-02-22,53,None,5.0,Debris Burning,250.00,D,27.090173,-97.788942,TX,2,Wednesday,NaN,kenedy,48261,2,48,Accident,2.625,17.25,4.25,9.875,34.00,62555.875,22581.994141,10454.875,204.625
690329,274403,2001-04-14,104,1540,7.0,Arson,1.00,B,47.088800,-95.836400,MN,4,Saturday,1540.0,becker,27005,1,27,Arson,2.000,284.00,368.50,30.500,685.00,64151.500,28786.000000,8523.000,131.000


In [1]:
# ran in a py script 
#try more features
# x6 = np.array(combined[['Month', 'LATITUDE', 'LONGITUDE', 'Prev_Lightning_Fires','Prev_Accident_Fires',
#        'Prev_Arson_Fires', 'Prev_Other_Fires', 'Prev_Fires_at_Location', 'Prev_Fires_Same_Month', 'Prev_1_fires2', 'Prev_2_Fires', 'Prev_3_fires']])
# y6 = np.array(combined['CLASS'])
# weights={1:1, 2:5, 3:41}
# rf6 = RandomForestClassifier(n_estimators=1000, n_jobs=14, max_depth=4, class_weight=weights, min_samples_leaf=2, verbose=1)
# %time cv_features(rf6, x6, y6)

NameError: name 'np' is not defined

In [10]:
# Score rf6
with open('fires_rf6.pkl', 'rb') as picklefile:
    rf6 = pickle.load(picklefile)


In [15]:
#filter out holdout sets
remaining_evals = other_stats[other_stats['FOD_ID'].isin(evals['FOD_ID'])]
evals2 = pd.concat([evals, remaining_evals.iloc[:,-5:]], axis=1)
remaining_recents = other_stats[other_stats['FOD_ID'].isin(recents['FOD_ID'])]
recents2 = pd.concat([recents, remaining_recents.iloc[:,-5:]], axis=1)
print(evals2.shape, recents2.shape)

(10000, 28) (10000, 28)


In [16]:
eval_X = np.array(evals2[['Month', 'LATITUDE', 'LONGITUDE', 'Prev_Lightning_Fires','Prev_Accident_Fires',
       'Prev_Arson_Fires', 'Prev_Other_Fires', 'Prev_Fires_at_Location', 'Prev_Fires_Same_Month', 'Prev_1_fires2', 'Prev_2_Fires', 'Prev_3_fires']])
eval_y = np.array(evals2['CLASS'])

print(confusion_matrix(eval_y, rf6.predict(eval_X), labels=[1,2,3]))
print(classification_report(eval_y, rf6.predict(eval_X), labels=[1,2,3]))

[Parallel(n_jobs=14)]: Done  22 tasks      | elapsed:    0.0s
[Parallel(n_jobs=14)]: Done 172 tasks      | elapsed:    0.1s
[Parallel(n_jobs=14)]: Done 422 tasks      | elapsed:    0.1s
[Parallel(n_jobs=14)]: Done 772 tasks      | elapsed:    0.3s
[Parallel(n_jobs=14)]: Done 1000 out of 1000 | elapsed:    0.3s finished
[Parallel(n_jobs=14)]: Done  22 tasks      | elapsed:    0.0s


[[7670 1011   90]
 [ 781  373   16]
 [  36   10   13]]


[Parallel(n_jobs=14)]: Done 172 tasks      | elapsed:    0.1s
[Parallel(n_jobs=14)]: Done 422 tasks      | elapsed:    0.1s
[Parallel(n_jobs=14)]: Done 772 tasks      | elapsed:    0.3s


             precision    recall  f1-score   support

          1       0.90      0.87      0.89      8771
          2       0.27      0.32      0.29      1170
          3       0.11      0.22      0.15        59

avg / total       0.82      0.81      0.81     10000



[Parallel(n_jobs=14)]: Done 1000 out of 1000 | elapsed:    0.3s finished


In [17]:
recent_X = np.array(recents2[['Month', 'LATITUDE', 'LONGITUDE', 'Prev_Lightning_Fires','Prev_Accident_Fires',
       'Prev_Arson_Fires', 'Prev_Other_Fires', 'Prev_Fires_at_Location', 'Prev_Fires_Same_Month', 'Prev_1_fires2', 'Prev_2_Fires', 'Prev_3_fires']])
recent_y = np.array(recents2['CLASS'])

print(confusion_matrix(recent_y, rf6.predict(recent_X), labels=[1,2,3]))
print(classification_report(recent_y, rf6.predict(recent_X), labels=[1,2,3]))

[Parallel(n_jobs=14)]: Done  22 tasks      | elapsed:    0.0s
[Parallel(n_jobs=14)]: Done 172 tasks      | elapsed:    0.1s
[Parallel(n_jobs=14)]: Done 422 tasks      | elapsed:    0.1s
[Parallel(n_jobs=14)]: Done 772 tasks      | elapsed:    0.3s
[Parallel(n_jobs=14)]: Done 1000 out of 1000 | elapsed:    0.3s finished
[Parallel(n_jobs=14)]: Done  22 tasks      | elapsed:    0.0s


[[8235  694   83]
 [ 724  211   25]
 [  16    5    7]]


[Parallel(n_jobs=14)]: Done 172 tasks      | elapsed:    0.1s
[Parallel(n_jobs=14)]: Done 422 tasks      | elapsed:    0.1s


             precision    recall  f1-score   support

          1       0.92      0.91      0.92      9012
          2       0.23      0.22      0.23       960
          3       0.06      0.25      0.10        28

avg / total       0.85      0.85      0.85     10000



[Parallel(n_jobs=14)]: Done 772 tasks      | elapsed:    0.3s
[Parallel(n_jobs=14)]: Done 1000 out of 1000 | elapsed:    0.3s finished


There is some signal! Much better than the 0.01 I thought I was getting. Also the results are very close to my validation results so it is generalizable. Next is getting the weather data and be more selective in adding the features so I can select the most important features.